In [1]:
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime     
import mysql.connector 

In [2]:


now = datetime.now()
dt = now.strftime("%d%m%Y_%H%M%S")

try:
    host = "localhost"
    user  = "root"
    pswd = input("Enter MySQL password : ")


    MyDB = mysql.connector.connect(host = host, user = user, password = pswd)

    MyCursor = MyDB.cursor()

    MyCursor.execute("create database IF NOT EXISTS MINeD")

    MyDB = mysql.connector.connect(host = host, user = user, password = pswd, database = "MINeD")
    MyCursor = MyDB.cursor()

    statement = str("CREATE TABLE IF NOT EXISTS Emp_Detect_"+dt+" (Employee_Code int(45) NOT NULL PRIMARY KEY, Identification_Time datetime default now())")
    MyCursor.execute(statement)

except Exception as e:
    print("An error occured...")
    print(e)





In [3]:
path = 'dataset1/img'

In [4]:
images = []
EmpNumber = []
mylist = os.listdir(path)

In [5]:
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    EmpNumber.append(os.path.splitext(cl)[0])

In [6]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)[0]
        encodeList.append(encoded_face)
    return encodeList

In [7]:
encoded_face_train = findEncodings(images)


if u want to capture from live webcam write the below line


cap = cv2.VideoCapture(0)

In [ ]:
# take pictures from video
# if u want to capture from live webcam write the below line
# cap = cv2.VideoCapture(0)
cap  = cv2.VideoCapture('dataset1/Video_clip/VID_20230302_170157.mp4') #

while True: 
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    faces_in_frame = face_recognition.face_locations(imgS)
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

    for encode_face, faceloc in zip(encoded_faces,faces_in_frame):
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)
        
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
        matchIndex = np.argmin(faceDist)
        

        if matches[matchIndex]:
            name = EmpNumber[matchIndex].upper().lower()
            y1,x2,y2,x1 = faceloc
            
            y1, x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img, (x1,y2-35),(x2,y2), (0,255,0), cv2.FILLED)
            cv2.putText(img,name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            try:
                SQLStatement = "INSERT IGNORE INTO Emp_Detect_"+dt+" (Employee_Code) VALUES (%s)"
                MyCursor.execute(SQLStatement, (name, ))
                MyDB.commit()
            
            except Exception as e:
                print("An Error Occured while Entering data into the MySQL table.")
                
    cv2.imshow('MINed Hackathon Track 5 J K L Team : Dronacharyans ', img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
        

cv2.destroyAllWindows()
cap.release()
print("program ended")


